In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [2]:
!wget --no-check-certificate \
    "https://github.com/EchaAz/Eye-Disease-Detections/archive/refs/heads/main.zip" \
    -O "/tmp/eye-disease.zip"


zip_ref = zipfile.ZipFile('/tmp/eye-disease.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2022-06-08 08:40:44--  https://github.com/EchaAz/Eye-Disease-Detections/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/EchaAz/Eye-Disease-Detections/zip/refs/heads/main [following]
--2022-06-08 08:40:44--  https://codeload.github.com/EchaAz/Eye-Disease-Detections/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/eye-disease.zip’

/tmp/eye-disease.zi     [    <=>             ]  15.77M  22.0MB/s    in 0.7s    

2022-06-08 08:40:45 (22.0 MB/s) - ‘/tmp/eye-disease.zip’ saved [16538701]



In [3]:
# Set the Path for your training and test data ser
PATH = "/tmp/Eye-Disease-Detections-main/Capstone_Eye_Diseases"
data_dir_list = os.listdir(PATH)
data_dir_list

['test', 'Train']

In [4]:
# Define the train and test base directories
train_dir = PATH + '/Train'
validation_dir = PATH + '/test'

In [5]:
dir_list = os.listdir(train_dir)
print(dir_list)
dir_list2 = os.listdir(validation_dir)
print(dir_list2)

['normal', 'conjunctivitis', 'ocular_herpes', 'cataract']
['normal', 'conjunctivitis', 'ocular_herpes', 'cataract']


In [6]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   horizontal_flip = True)
  
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=8, 
                                                      class_mode='categorical',
                                                      target_size=(150, 150))
  
  validation_datagen = ImageDataGenerator(rescale = 1./255.)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=8, 
                                                                class_mode='categorical',
                                                                target_size=(150, 150))
  
  return train_generator, validation_generator

In [7]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 602 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [8]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-06-08 08:40:47--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 173.194.196.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   208MB/s    in 0.4s    

2022-06-08 08:40:48 (208 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [9]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
def create_pre_trained_model(local_weights_file):
  pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  for layer in pre_trained_model.layers:
      layer.trainable = False

  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

In [ ]:
# Define a Callback class that stops training once validation accuracy reaches 70%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.70):
      print("\nReached 70% validation accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
def output_of_last_layer(pre_trained_model):
  last_desired_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)
  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

last layer output shape:  (None, 7, 7, 768)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")


In [ ]:
def create_final_model(pre_trained_model, last_output):
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.5)(x)
  # Add a final softmax layer for classification
  x = layers.Dense(4, activation='softmax')(x)   

  # Create the complete model by using the Model class
  model = Model(pre_trained_model.input, x) 

  # Compile the model
  model.compile(optimizer = RMSprop(learning_rate=0.0001), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
# Save your model in a variable
model = create_final_model(pre_trained_model, last_output)

In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 50,
                    verbose = 1,
                    callbacks=callbacks)

Epoch 1/50
76/76 [==============================] - 5s 69ms/step - loss: 0.2610 - accuracy: 0.9153 - val_loss: 2.2572 - val_accuracy: 0.6200
Epoch 2/50
76/76 [==============================] - 5s 67ms/step - loss: 0.2495 - accuracy: 0.9269 - val_loss: 1.7886 - val_accuracy: 0.7000
Epoch 3/50
76/76 [==============================] - 5s 71ms/step - loss: 0.2205 - accuracy: 0.9302 - val_loss: 2.2650 - val_accuracy: 0.6200
Epoch 4/50
76/76 [==============================] - 5s 67ms/step - loss: 0.2373 - accuracy: 0.9336 - val_loss: 1.8663 - val_accuracy: 0.6600
Epoch 5/50
76/76 [==============================] - 5s 69ms/step - loss: 0.2459 - accuracy: 0.9169 - val_loss: 1.9511 - val_accuracy: 0.5800
Epoch 6/50
76/76 [==============================] - 5s 67ms/step - loss: 0.2255 - accuracy: 0.9186 - val_loss: 1.9753 - val_accuracy: 0.6400
Epoch 7/50
76/76 [==============================] - 5s 67ms/step - loss: 0.2119 - accuracy: 0.9269 - val_loss: 1.6207 - val_accuracy: 0.7000
Epoch 8/50
76

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

Saving EYE(!).jpg to EYE(!).jpg
EYE(!).jpg
[[0. 1. 0. 0.]]
